In [2]:
#!pip install geopandas
#!pip install flask-ngrok
#!pip install flask-cors

import pandas as pd
import geopy.distance
import geopandas as gpd
import json
from shapely.geometry import Point
import numpy as np
import math
import random
from shapely.ops import cascaded_union

In [37]:
dit_df = pd.read_excel("b7fdff5a1a993b1e.xlsx")
dit_df

,house_district,person_gender,Возрастной класс,person_exemption_group,sum
0,Академический,f,18-25,Многодетные,100
1,Академический,f,18-25,NaN,3895
2,Академический,f,25-45,Беременность,318
3,Академический,f,25-45,Инвалид,159
4,Академический,f,25-45,Малоимущие,3
...,...,...,...,...,...
13566,Ясенево,m,90<,Ветеран труда,8
13567,Ясенево,m,90<,Пенсионер,80
13568,Ясенево,m,<7,Дети инвалиды,56
13569,Ясенево,m,<7,Многодетные,121


In [38]:
mo_df = gpd.read_file('mo.geojson')
mo_df["NAME"] = [elem.replace("ё", "е").replace('"', '') for elem in mo_df["NAME"].tolist()]
mo_df

,NAME,OKATO,OKTMO,NAME_AO,OKATO_AO,ABBREV_AO,TYPE_MO,geometry
0,Киевский,45298555,45945000,Троицкий,45298000,Троицкий,Поселение,"MULTIPOLYGON (((36.80310 55.44083, 36.80319 55..."
1,Филевский Парк,45268595,45328000,Западный,45268000,ЗАО,Муниципальный округ,"POLYGON ((37.42765 55.74821, 37.42849 55.74875..."
2,Новофедоровское,45298567,45954000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((36.80357 55.45162, 36.80451 55.46551..."
3,Роговское,45298575,45956000,Троицкий,45298000,Троицкий,Поселение,"POLYGON ((36.93724 55.24139, 36.93726 55.24161..."
4,Мосрентген,45297568,45953000,Новомосковский,45297000,Новомосковский,Поселение,"POLYGON ((37.43956 55.62731, 37.44018 55.63042..."
...,...,...,...,...,...,...,...,...
141,Ивановское,45263567,45306000,Восточный,45263000,ВАО,Муниципальный округ,"POLYGON ((37.79981 55.76232, 37.79981 55.76243..."
142,Косино-Ухтомский,45263573,45308000,Восточный,45263000,ВАО,Муниципальный округ,"POLYGON ((37.83602 55.70978, 37.83620 55.71008..."
143,Новокосино,45263579,45310000,Восточный,45263000,ВАО,Муниципальный округ,"POLYGON ((37.84042 55.73049, 37.84063 55.73196..."
144,Некрасовка,45290574,45391000,Юго-Восточный,45290000,ЮВАО,Муниципальный округ,"POLYGON ((37.90613 55.70626, 37.90701 55.70699..."


In [39]:
audience_final_df = dit_df.merge(mo_df, left_on="house_district", right_on="NAME")[["house_district", "person_gender", "Возрастной класс", "person_exemption_group", "sum", "geometry"]]
audience_final_df = audience_final_df.dropna()

In [8]:
df_624 = pd.read_csv("624.csv", sep=";")[["STATION", "X", "Y"]]
df_624["X"] = df_624.X.apply(lambda x: x.replace(",", "."))
df_624["Y"] = df_624.Y.apply(lambda x: x.replace(",", "."))
df_624["X"] = df_624["X"].astype("float")
df_624["Y"] = df_624["Y"].astype("float")
df_624 = df_624.groupby("STATION").mean().reset_index()
df_624

,STATION,X,Y
0,Авиамоторная,37.716940,55.751536
1,Автозаводская,37.657538,55.707810
2,Академическая,37.573470,55.687864
3,Александровский сад,37.610477,55.752287
4,Алексеевская,37.638699,55.807853
...,...,...,...
170,Щёлковская,37.798567,55.809693
171,Электрозаводская,37.705301,55.782116
172,Юго-западная,37.483216,55.663656
173,Южная,37.608856,55.622337


In [9]:
df_metro_set = pd.read_csv("metro_set.csv")
df_metro_set = df_metro_set[["station_nm", "v_cnt"]].groupby("station_nm").sum().reset_index()
df_metro_set["v_cnt"] = df_metro_set["v_cnt"]/(12*7)
df_metro_set = df_metro_set.round()
df_metro_set

,station_nm,v_cnt
0,Авиамоторная,49953.0
1,Автозаводская,59468.0
2,Академическая,38403.0
3,Александровский сад,22443.0
4,Алексеевская,34925.0
...,...,...
230,Щёлковская,73415.0
231,Электрозаводская,34266.0
232,Юго-Западная,55627.0
233,Южная,25855.0


In [10]:
metro_final_df = df_624.merge(df_metro_set, left_on="STATION", right_on="station_nm")[["STATION", "X", "Y", "v_cnt"]]
metro_final_df

,STATION,X,Y,v_cnt
0,Авиамоторная,37.716940,55.751536,49953.0
1,Автозаводская,37.657538,55.707810,59468.0
2,Академическая,37.573470,55.687864,38403.0
3,Александровский сад,37.610477,55.752287,22443.0
4,Алексеевская,37.638699,55.807853,34925.0
...,...,...,...,...
152,Щукинская,37.464373,55.808590,43960.0
153,Щёлковская,37.798567,55.809693,73415.0
154,Электрозаводская,37.705301,55.782116,34266.0
155,Южная,37.608856,55.622337,25855.0


In [11]:
metro_stations_lons = metro_final_df.X.tolist()
metro_stations_lats = metro_final_df.Y.tolist()

In [12]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [14]:
DIT_DATA_df = pd.read_csv("DIT_DATA_AGGRS.csv")
DIT_DATA_df = DIT_DATA_df[["customers_cnt_total_sum", "customers_cnt_long_sum", "customers_cnt_work_sum", "customers_cnt_loc_sum", "lon", "lat"]].groupby(["lon", "lat"]).sum().reset_index()
DIT_DATA_lons = DIT_DATA_df.lon.tolist()
DIT_DATA_lats = DIT_DATA_df.lat.tolist()
DIT_DATA_df["customers_cnt_total_sum"] = DIT_DATA_df["customers_cnt_total_sum"]/60
DIT_DATA_df["customers_cnt_total_sum"] = DIT_DATA_df["customers_cnt_total_sum"].round()

DIT_DATA_df["customers_cnt_long_sum"] = DIT_DATA_df["customers_cnt_long_sum"]/60
DIT_DATA_df["customers_cnt_long_sum"] = DIT_DATA_df["customers_cnt_long_sum"].round()

DIT_DATA_df["customers_cnt_work_sum"] = DIT_DATA_df["customers_cnt_work_sum"]/60
DIT_DATA_df["customers_cnt_work_sum"] = DIT_DATA_df["customers_cnt_work_sum"].round()

DIT_DATA_df["customers_cnt_loc_sum"] = DIT_DATA_df["customers_cnt_loc_sum"]/60
DIT_DATA_df["customers_cnt_loc_sum"] = DIT_DATA_df["customers_cnt_loc_sum"].round()
DIT_DATA_df

,lon,lat,customers_cnt_total_sum,customers_cnt_long_sum,customers_cnt_work_sum,customers_cnt_loc_sum
0,37.372695,55.745245,415.0,292.0,67.0,215.0
1,37.372709,55.740753,539.0,359.0,61.0,391.0
2,37.372724,55.736262,518.0,378.0,124.0,310.0
3,37.372739,55.731770,1109.0,747.0,407.0,443.0
4,37.372753,55.727279,1978.0,1546.0,445.0,1027.0
...,...,...,...,...,...,...
4344,37.851375,55.848112,38.0,37.0,0.0,36.0
4345,37.851415,55.852603,54.0,53.0,0.0,52.0
4346,37.851456,55.857094,8.0,7.0,0.0,2.0
4347,37.851496,55.861586,2.0,2.0,1.0,0.0


In [15]:
def textAddressToLonLat(address):

  url= "https://geocode-maps.yandex.ru/1.x/?apikey="
  api_key_ = "cb2abc98-8052-4f76-bec3-535142b021a8"
  r = requests.get(url+api_key_+"&format=json&geocode="+address)
  json_ = json.loads(r.content)

  point = json_["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["Point"]["pos"].split()
  lon = float(point[0])
  lat = float(point[1])

  return lon, lat

In [33]:
def get_soc_dem_data(lon, lat):

  #audience
  mo_s = audience_final_df[['house_district', 'geometry']].drop_duplicates() 
  for i, polygon in enumerate(mo_s.geometry.tolist()):
    if Point(lon, lat).within(polygon):
      name = mo_s.house_district.tolist()[i]

      soc_dem_dict_ = {}

      audience_final_df_short = audience_final_df[audience_final_df["house_district"] == name]
     
      person_gender_df = audience_final_df_short[["person_gender", "sum"]].groupby("person_gender").sum().reset_index()
      soc_dem_dict_["sex"] = []
      soc_dem_dict_["sex"].append({"code": "f", "name": "Жен", "value": person_gender_df[person_gender_df["person_gender"]=="f"]["sum"].iloc[0]})
      soc_dem_dict_["sex"].append({"code": "m", "name": "Муж", "value": person_gender_df[person_gender_df["person_gender"]=="m"]["sum"].iloc[0]})
      soc_dem_dict_["total"] = person_gender_df["sum"].sum()

      age_class_df = audience_final_df_short[["Возрастной класс", "sum"]].groupby("Возрастной класс").sum().reset_index()
      soc_dem_dict_["age"] = []
      soc_dem_dict_["age"].append({"code": "", "name": "<7", "value": age_class_df[age_class_df["Возрастной класс"]=="<7"]["sum"].iloc[0]})
      soc_dem_dict_["age"].append({"code": "", "name": "7-18", "value": age_class_df[age_class_df["Возрастной класс"]=="7-18"]["sum"].iloc[0]})
      soc_dem_dict_["age"].append({"code": "", "name": "18-25", "value": age_class_df[age_class_df["Возрастной класс"]=="18-25"]["sum"].iloc[0]})
      soc_dem_dict_["age"].append({"code": "", "name": "25-45", "value": age_class_df[age_class_df["Возрастной класс"]=="25-45"]["sum"].iloc[0]})
      soc_dem_dict_["age"].append({"code": "", "name": "45-60", "value": age_class_df[age_class_df["Возрастной класс"]=="45-60"]["sum"].iloc[0]})
      soc_dem_dict_["age"].append({"code": "", "name": "60-75", "value": age_class_df[age_class_df["Возрастной класс"]=="60-75"]["sum"].iloc[0]})
      soc_dem_dict_["age"].append({"code": "", "name": "75-90", "value": age_class_df[age_class_df["Возрастной класс"]=="75-90"]["sum"].iloc[0]})
      soc_dem_dict_["age"].append({"code": "", "name": "90<", "value": age_class_df[age_class_df["Возрастной класс"]=="90<"]["sum"].iloc[0]})


      person_exemption_group_df = audience_final_df_short[["person_exemption_group", "sum"]].groupby("person_exemption_group").sum().reset_index()
      soc_dem_dict_["soc"] = []
      soc_dem_dict_["soc"].append({"code": "", "name": "Многодетные", "value": person_exemption_group_df[person_exemption_group_df["person_exemption_group"]=="Многодетные"]["sum"].iloc[0]})
      soc_dem_dict_["soc"].append({"code": "", "name": "Беременность", "value": person_exemption_group_df[person_exemption_group_df["person_exemption_group"]=="Беременность"]["sum"].iloc[0]})
      soc_dem_dict_["soc"].append({"code": "", "name": "Инвалид", "value": person_exemption_group_df[person_exemption_group_df["person_exemption_group"]=="Инвалид"]["sum"].iloc[0]})
      soc_dem_dict_["soc"].append({"code": "", "name": "Малоимущие", "value": person_exemption_group_df[person_exemption_group_df["person_exemption_group"]=="Малоимущие"]["sum"].iloc[0]})
      soc_dem_dict_["soc"].append({"code": "", "name": "Пенсионер", "value": person_exemption_group_df[person_exemption_group_df["person_exemption_group"]=="Пенсионер"]["sum"].iloc[0]})
      soc_dem_dict_["soc"].append({"code": "", "name": "Ветеран труда", "value": person_exemption_group_df[person_exemption_group_df["person_exemption_group"]=="Ветеран труда"]["sum"].iloc[0]})
      soc_dem_dict_["soc"].append({"code": "", "name": "Дети инвалиды", "value": person_exemption_group_df[person_exemption_group_df["person_exemption_group"]=="Дети инвалиды"]["sum"].iloc[0]})
      soc_dem_dict_["soc"].append({"code": "", "name": "Молодые родители", "value": person_exemption_group_df[person_exemption_group_df["person_exemption_group"]=="Молодые родители"]["sum"].iloc[0]})


  return soc_dem_dict_

In [17]:
def get_metro_data(lon, lat):
    metro_dists = []
    return_ = []
    for i in range(len(metro_stations_lons)):
      metro_dists.append(geopy.distance.vincenty((lat, lon), (metro_stations_lats[i], metro_stations_lons[i])).km)
    metro_final_df["dist"] = metro_dists
    head_ = metro_final_df[["STATION", "v_cnt", "dist"]].sort_values("dist").head(3)

    for row in head_.iterrows():
      return_.append({"color": "", "name": row[1].STATION, "count": row[1].v_cnt, "dist": row[1].dist})
    
    return return_
    #return json.loads(metro_final_df[["STATION", "v_cnt", "dist"]].sort_values("dist").head(3).to_json(orient="records", force_ascii=False))

In [121]:
business_df = pd.read_csv("business.csv")
business_lons = business_df.lon.tolist()
business_lats = business_df.lat.tolist()

total_in_moscow_df = business_df[["type"]].groupby("type").size().reset_index()
total_in_moscow_df.columns = ["type", "count"]
total_in_moscow_df

,type,count
0,atm,31
1,bar,505
2,cafe,3283
3,cinema,104
4,clinic,780
5,fuel,445
6,nightclub,79
7,pharmacy,2243
8,restaurant,2370
9,vending_machine,98


In [31]:
mo_df = gpd.read_file('mo.geojson').to_crs({'init': 'epsg:6933'})
mo_df["NAME"] = [elem.replace("ё", "е").replace('"', '') for elem in mo_df["NAME"].tolist()]
density_df = audience_final_df[["house_district", "sum", "geometry"]]
density_df["geometry"] = density_df.geometry.apply(lambda x: x.area/10**6)
density_df.columns = ["house_district", "sum", "area"]
density_df = density_df.groupby(["house_district", "area"]).sum().reset_index()
mo_df = gpd.read_file('mo.geojson')
density_df = density_df.merge(mo_df, left_on="house_district", right_on="NAME")[["house_district", "area", "sum", "geometry"]]
density_df

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,house_district,area,sum,geometry
0,Академический,5.709313,27357,"POLYGON ((37.55226 55.68403, 37.55232 55.68408..."
1,Алексеевский,5.239039,19765,"POLYGON ((37.63499 55.79714, 37.63552 55.79965..."
2,Алтуфьевский,3.295936,14596,"POLYGON ((37.56740 55.88730, 37.57245 55.88724..."
3,Арбат,2.084146,7287,"POLYGON ((37.57206 55.75202, 37.57396 55.75256..."
4,Аэропорт,4.666697,19850,"POLYGON ((37.51101 55.80608, 37.51137 55.80647..."
...,...,...,...,...
127,Южное Тушино,7.892683,27861,"POLYGON ((37.39182 55.84723, 37.39187 55.84762..."
128,Южнопортовый,4.583960,18601,"POLYGON ((37.66069 55.73070, 37.66082 55.73073..."
129,Якиманка,4.805900,7009,"POLYGON ((37.57761 55.71421, 37.57875 55.71478..."
130,Ярославский,8.357428,24102,"POLYGON ((37.66347 55.84080, 37.66458 55.84164..."


In [111]:
from collections import Counter
Counter(business_df["type"].tolist()).most_common()[-1][0]

'bar'

In [122]:
def get_businesses(lon, lat, radius):

  amenities = []
  report = {}
  dists = []
  coords_1 = (lat, lon)

  dictionary = {"bar": "Бары", 
                  "restaurant": "Рестораны",
                  "cafe": "Кафе",
                  "pharmacy": "Аптеки",
                  "fuel": "АЗС",
                  "atm": "АТМ-автоматы",
                  "clinic": "Клиники",
                  "cinema": "Кинотеатры",
                  "nightclub": "Ночные клубы",
                  "vending_machine": "Вендинговые автоматы"}

  for i, poly in enumerate(density_df.geometry.tolist()):
    if Point(lon, lat).within(poly):
      report["district"] = density_df.house_district.tolist()[i]
      people_here = density_df["sum"].tolist()[i]
      area_here = density_df["area"].tolist()[i]
      report["people_density_here"] = people_here/area_here
      report["people_density_in_Moscow"] = 4554
      report["people_in_sector"] = round((people_here/area_here)*3.14*(radius**2))

  for i in range(len(business_lons)):
    dists.append(geopy.distance.vincenty(coords_1, (business_lats[i], business_lons[i])).km)

  business_df_copy = business_df.copy()
  business_df_copy["dist"] = dists
  new_business_df = business_df_copy[business_df_copy['dist']<radius]

  for amenity_ in business_df_copy.type.unique():
    object_ = {}
    object_["code"] = amenity_
    object_["name"] = dictionary[amenity_]

    #число объектов в радиусе
    object_["countObject"] = {}
    object_["countObject"]["value"] = new_business_df[new_business_df["type"]==amenity_].shape[0]
    max_value = Counter(new_business_df["type"].tolist()).most_common(1)[0][1]
    object_["countObject"]["percent"] = round((new_business_df[new_business_df["type"]==amenity_].shape[0]/max_value)*100)

    #число объектов в Москве
    object_["allObject"] = {}
    object_["allObject"]["value"] = total_in_moscow_df[total_in_moscow_df["type"]==amenity_]["count"].iloc[0]
    max_value = total_in_moscow_df["count"].max()
    object_["allObject"]["percent"] = round((total_in_moscow_df[total_in_moscow_df["type"]==amenity_]["count"].iloc[0]/max_value)*100)

    #число людей на 1 объект в радиусе
    object_["countPeople"] = {}
    object_["countPeople"]["value"] = (people_here/area_here)*3.14*(radius**2)/new_business_df[new_business_df["type"]==amenity_].shape[0]
    max_value = (people_here/area_here)*3.14*(radius**2)/new_business_df[new_business_df["type"]==Counter(business_df["type"].tolist()).most_common()[-1][0]].shape[0]
    object_["countPeople"]["percent"] = object_["countPeople"]["value"]/max_value

    #среднее число людей на 1 объект в Москве
    object_["averagePeople"] = {}
    object_["averagePeople"]["value"] = 11673511/total_in_moscow_df[total_in_moscow_df["type"]==amenity_]["count"].iloc[0]
    max_value = 11673511/total_in_moscow_df[total_in_moscow_df["type"]==Counter(business_df["type"].tolist()).most_common()[-1][0]]["count"].iloc[0]
    object_["averagePeople"]["percent"] = object_["averagePeople"]["value"]/max_value


    object_["differencePeople"] = {}
    differencePeople = round((object_["countPeople"]["value"]/object_["averagePeople"]["value"]-1)*100, 2)
    if differencePeople > 0:
      object_["differencePeople"]["value"] = "+"+str(differencePeople)+"%"
      object_["differencePeople"]["percent"] = differencePeople
      object_["differencePeople"]["color"] = "green"
    else:
      object_["differencePeople"]["value"] = str(differencePeople)+"%"
      object_["differencePeople"]["percent"] = differencePeople
      object_["differencePeople"]["color"] = "red"
    
    amenities.append(object_)

  report["businesses"] = amenities

  return report

In [23]:
def get_dit_data(lon, lat):
    DIT_DATA_dists = []
    for i in range(len(DIT_DATA_lons)):
      DIT_DATA_dists.append(geopy.distance.vincenty((lat, lon), (DIT_DATA_lats[i], DIT_DATA_lons[i])).km)
    DIT_DATA_df["dists"] = DIT_DATA_dists
    return json.loads(DIT_DATA_df[DIT_DATA_df["dists"] == DIT_DATA_df["dists"].min()][["customers_cnt_total_sum", "customers_cnt_long_sum", "customers_cnt_work_sum", "customers_cnt_loc_sum"]].to_json(orient="records", force_ascii=False))

In [24]:
polygons = mo_df.geometry.tolist()
boundary = gpd.GeoSeries(cascaded_union(polygons))

In [25]:
#для Y (~55) 500 м это примерно 0,0044909
#для координаты X (~37) 500 м это примерно 0,00798375

def synthetic_points_in_sector(center_lon, center_lat, n):

  points = []
  lo_1 = center_lon-0.00798375
  lo_2 = center_lon+0.00798375
  la_1 = center_lat-0.0044909
  la_2 = center_lat+0.0044909

  for i in range(n):
    lon = random.uniform(lo_1, lo_2)
    lat = random.uniform(la_1, la_2)
    points.append({"lat": lat, "lon": lon})
  
  return(points)

In [26]:
def point_in_boundary(x):
  return Point(x['lon'], x['lat']).within(boundary[0])

In [27]:
def get_heat_map():

  df = pd.read_csv("DIT_DATA_AGGRS.csv")
  df = df[["customers_cnt_total_sum", "customers_cnt_long_sum", "customers_cnt_work_sum", "customers_cnt_loc_sum", "lon", "lat"]].groupby(["lon", "lat"]).sum().reset_index()
  new_df = df[["customers_cnt_total_sum", "lon", "lat"]]
  max_ = new_df.customers_cnt_total_sum.max()
  new_df["customers_cnt_total_sum"] = new_df.customers_cnt_total_sum.apply(lambda x: math.ceil(x*10/max_))
  final_points = []

  for row in new_df.iterrows():
    for elem in synthetic_points_in_sector(row[1].lon, row[1].lat, int(row[1].customers_cnt_total_sum)):
      final_points.append(elem)

  final_points = [x for x in final_points if point_in_boundary(x)]
  return final_points

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import request
import requests
import json
from flask import jsonify
from flask_cors import CORS, cross_origin


app = Flask(__name__)
cors = CORS(app)
run_with_ngrok(app)
@app.route("/api/v1/")
def home():

    if request.args.get('action') == "address.getData":

      total_json = {}

      lon, lat = textAddressToLonLat(request.args.get('address'))
      radius = int(request.args.get('radius'))

      total_json["socDem"] = get_soc_dem_data(lon, lat)
      total_json["transport"] = get_metro_data(lon, lat)
      total_json["bestBusiness"] = get_businesses(lon, lat, radius)
      total_json["DIT_DATA"] = get_dit_data(lon, lat)

      total_json["address"] = {}
      total_json["address"]["id"] = 1
      total_json["address"]["name"] = request.args.get('address')
      total_json["address"]["map"] = [lat, lon]

      return json.dumps(total_json, cls=NpEncoder, ensure_ascii=False)


    elif request.args.get('action') == "map.getHeatMap":
      return json.dumps(get_heat_map())


    elif request.args.get('action') == "map.getRivalList":

      business_type = request.args.get('business_type')
      business_df_copy = business_df.copy()

      return json.dumps(json.loads(business_df_copy[business_df_copy["type"] == business_type].to_json(orient="records", force_ascii=False)), ensure_ascii=False)
    

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ec6ac934a4a9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [01/Nov/2020 08:06:11] "GET /api/v1/?action=address.getData&address=Южное%20Тушино&radius=3 HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 08:06:12] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2020 08:06:24] "GET /api/v1/?action=address.getData&address=Южное%20Тушино&radius=3 HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 08:06:34] "GET /api/v1/?action=address.getData&address=Южное%20Тушино&radius=3 HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 08:06:35] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [01/Nov/2020 08:06:56] "GET /api/v1/?action=address.getData&address=Дубнинская+улица%2C+дом+31&radius=3 HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 08:07:21] "GET /api/v1/?action=address.getData&address=Дубнинская+улица%2C+дом+31&radius=3 HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 08:11:26] "GET /api/v1/?action=address.getData&address=Дубнинская+улица%2C+дом+31&radius=3 HTTP/1.1" 200 -
